In [1]:
import socket
import struct

# API

In [2]:
INFO = b"UR001!"
ID = b"UL!"
FIRMWARE = b"UN!"
POWER_ON = b"UK001!"
POWER_OFF = b"UK004!"

def set_grill_temp(temp_f):
    if not (150 <= temp_f <= 450):
        raise ValueError("invalid temperature")
    return b"UT???!".replace(b"???", format(temp_f, "03d").encode("ascii"))

def set_probe_temp(temp_f):
    if not (50 <= temp_f <= 220):
        raise ValueError("invalid temperature")
    return b"UF{}!"

def broadcast():
    return "UH{}{}{}{}{}!".format(0, len(ssid), ssid, len(password), password)

In [4]:
set_grill_temp(234)

b'UT234!'

In [5]:
import base64
import fcntl
import ipaddress
import socket
import subprocess

In [7]:
socket.if_nameindex()

[(1, 'lo'),
 (2, 'wwan0'),
 (3, 'wlp58s0'),
 (4, 'br-73fb78ea0798'),
 (5, 'br-8df54316e2a9'),
 (6, 'docker0'),
 (7, 'br-63aed09d3259'),
 (104, 'tailscale0'),
 (112, 'veth1825448'),
 (182, 'vetha38fb45'),
 (193, 'anbox0')]

In [8]:
iface = "wlp58s0"
socket.inet_ntoa(
    fcntl.ioctl(
        socket.socket(socket.AF_INET, socket.SOCK_DGRAM),
        35099,
        struct.pack('256s', iface.encode("ascii"))
    )[20:24]
)

'255.255.255.0'

In [9]:
class MAC:
    """MAC or MAC Prefix"""
    def __init__(self, address):
        self.raw = base64.b16decode(address.replace(":", ""), casefold=True)

    def __repr__(self):
        return f"{self.__class__.__name__}(address='{self}')"

    def __str__(self):
        return ":".join(hex(b)[2:] for b in self.raw)
    
    def __contains__(self, other):
        """Does this MAC prefix contain the provided MAC?"""
        return other.raw.startswith(self.raw)

In [10]:
x = MAC("7C:a7")
x in MAC("7c")

True

In [11]:
base64.b16decode("7C:a7:b0".replace(":", ""), casefold=True)

b'|\xa7\xb0'

In [14]:
def local_macs(cidr):
    network = ipaddress.IPv4Network(cidr)
    # attempt connection to everything to populate ARP
    subprocess.check_output(["nmap", "-sP", str(network)])
    
    # dump ARP table
    neighbors = subprocess.check_output(["ip", "neighbor", "show"], universal_newlines=True)
    
    # and parse output from `ip neigh`
    arp_pairs = []
    for line in neighbors.splitlines():
        ip, _dev, interface, result = line.split(" ", 3)
        result = result.strip()
        if "lladdr" not in result:
            # FAILED, INCOMPLETE, etc.
            continue
        # REACHABLE, STALE, DELAY, etc.
        _lladdr, mac, status = result.split()
        arp_pairs.append((ip, mac))

    return arp_pairs

In [15]:
macs = local_macs("192.168.42.0/24")

In [16]:
gmg_mac = "7c:a7:b0:af:4e:58"
mac_to_ip = {mac: ip for ip, mac in macs}

mac_to_ip[gmg_mac]

'192.168.42.245'

In [17]:
def send_command(command: bytes, ip, port=8080):
    address = (ip, port)
    with socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM) as sock:
        sock.connect(address)
        sock.send(command)
        return sock.recv(128)

In [22]:
send_command(ID, mac_to_ip[gmg_mac])[:5]

b'GMG12'

In [23]:
send_command(FIRMWARE, mac_to_ip[gmg_mac])[:5]

b'UNDB0'

In [24]:
send_command(INFO, mac_to_ip[gmg_mac])

b'UR%\x00Y\x02\x96\x00\x05\n\x142\x19\x19\x19\x19&\x00\x00\x00\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x03'

In [108]:
import base64
import struct

In [26]:
info_data = send_command(INFO, mac_to_ip[gmg_mac])
len(info_data)

36

In [27]:
info_data

b'UR%\x00Y\x02\x96\x00\x05\n\x142\x19\x19\x19\x19&\x00\x00\x00\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x03'

In [28]:
base64.b16encode(info_data)

b'5552250059029600050A14321919191926000000FFFFFFFF000000000000000001000003'

# Grill Data Format

In [33]:
spec = [
    ("2x", "?"), # always "UR"
    ("h", "grill_temp"),   # 2-3
    ("h", "probe1_temp"),  # 4-5
    ("h", "grill_target"), # 6-7
    ("x", "?"), # 8
    ("x", "?"), # 9
    ("x", "?"), # 10
    ("x", "?"), # 11
    ("x", "?"), # 12
    ("x", "?"), # 13
    ("x", "?"), # 14
    ("x", "?"), # 15
    ("h", "probe2_temp"),
    ("x", "?"),
    ("x", "?"),
    ("b", "curve_remain_time"),
    ("x", "?"),
    ("x", "?"),
    ("x", "?"),
    ("b", "warn_code"),
    ("x", "?"),
    ("x", "?"),
    ("x", "?"),
    ("h", "probe1_target"),
    ("b", "grill_state"), # 30
    ("b", "grill_mode"),
    ("b", "fire_state"),
    ("b", "fire_state_pct"),
    ("x", "?"),
    ("x", "?"),
]

In [34]:
def specunpack(spec, buf, byte_order=""):
    """
    Given a 'spec' for a structure which is a sequence of tuple pairs, each
    consisting of a struct format character and a 'name'. On unpacking 'buf',
    the names are used in the returned dictionary.

    Pad characters ('x') are ignored; the name in those pairs is irrelevant.
    """
    format_ = [byte_order]
    names = set()
    for part, name in spec:
        format_.append(part)
        if "x" in part:
            continue
        if name in names:
            raise ValueError(f"duplicate names: {name}")
        names.add(name)
    unpacked = struct.unpack("".join(format_), buf)
    output = {}
    for name, data in zip((name for part, name in spec if "x" not in part), unpacked):
        output[name] = data
    return output

In [35]:
specunpack(spec, info_data, byte_order="<")

{'grill_temp': 37,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 38,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

# Reverse Engineering Fields

In [36]:
import datetime
import subprocess

def xxd(buf):
    """call xxd on some binary"""
    p = subprocess.Popen(['xxd', "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout_data = p.communicate(input=data)[0].decode()
    print(stdout_data)

Nothing plugged in to probes, grill off

In [216]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
docunpack(spec, data)

2022-02-09T00:22:43.839540

00000000: 5552 2500 5902 9600 050a 1432 1919 1919  UR%.Y......2....
00000010: 5902 0000 ffff ffff 0000 0000 0000 0000  Y...............
00000020: 0100 0003                                ....



{'grill_temp': 37,
 'food_temp': 601,
 'grill_target': 150,
 'curve_remain_time': -1,
 'warn_code': 0,
 'food_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Probe 1 plugged in

In [218]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
docunpack(spec, data)

2022-02-09T00:24:16.204785

00000000: 5552 2500 3d00 9600 050a 1432 1919 1919  UR%.=......2....
00000010: 5902 0000 ffff ffff 0000 0000 0000 0000  Y...............
00000020: 0100 0003                                ....



{'grill_temp': 37,
 'food_temp': 61,
 'grill_target': 150,
 'curve_remain_time': -1,
 'warn_code': 0,
 'food_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Probe 1 unplugged, probe 2 plugged in

In [225]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
docunpack(spec, data)

2022-02-09T00:27:52.654772

00000000: 5552 2500 5902 9600 050a 1432 1919 1919  UR%.Y......2....
00000010: 2400 0000 ffff ffff 0000 0000 0000 0000  $...............
00000020: 0100 0003                                ....



{'grill_temp': 37,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 36,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}